# Debug

In [1]:
from utils import (
    train_regression_model,
    evaluate_model,
    load_simulated_regression_data,
)
X, y, y_true = load_simulated_regression_data(
    i=1,
    SNR=100,
    num_interact=2,
    order=3,
    n=1000,
    p = 50,
    feature_correlation=0,
    overlap=1,
    noise_type='Gaussian',
)

In [8]:
from irf.ensemble import wrf_reg as rfr
from irf.utils import (
    get_prevalent_interactions,
    visualize_impurity_decrease,
    visualize_prevalent_interactions,
    get_filtered_feature_paths
)
rf = rfr(
    n_jobs=16,
    n_estimators=300,
    max_depth=None,
    bootstrap=True,
)
rf.fit(
    X,
    y,
    keep_record=False,
    K=10,
)
#print(rf.n_paths)
min_support = rf.n_paths // 2 ** (3 + 1)
prevalence = get_prevalent_interactions(
    rf,
    impurity_decrease_threshold=0,
    min_support=min_support,
    signed=True,
    weight_scheme='depth',
    adjust_for_weights=True,
)
prevalence

OrderedDict([(((2, 'L'),), 0.4999999999999992),
             (((2, 'R'),), 0.499999999999981),
             (((2, 'L'), (3, 'L'), (4, 'L')), 0.12499999999997984),
             (((0, 'L'), (1, 'L'), (2, 'L')), 0.12499999999997856),
             (((2, 'L'), (4, 'L')), 0.21586056709286328),
             (((2, 'L'), (4, 'R')), 0.2158605670928489),
             (((1, 'R'), (2, 'L'), (4, 'R')), 0.09899904886880583),
             (((1, 'L'), (2, 'L')), 0.21011983235673515),
             (((1, 'R'), (2, 'L')), 0.21011983235673037),
             (((1, 'R'), (2, 'L'), (4, 'L')), 0.09560234069822764),
             (((2, 'L'), (3, 'R'), (4, 'L')), 0.09086056709288147),
             (((1, 'L'), (2, 'L'), (4, 'R')), 0.09077532132465116),
             (((1, 'L'), (2, 'L'), (4, 'L')), 0.08737861315407931),
             (((0, 'L'), (2, 'L'), (4, 'L')), 0.08551715691883213),
             (((0, 'R'), (1, 'L'), (2, 'L')), 0.08511983235675832),
             (((1, 'L'), (2, 'L'), (3, 'L')), 0.08419811248777

In [14]:
import numpy as np
order1 = sorted(prevalence.items(), 
    #key= lambda t: -np.log2(t[1]) / len(t[0]),
    key=lambda t: (2 ** (-len(t[0])) - t[1]) / (2 ** (-len(t[0])) * (1 - 2 ** (-len(t[0])))) ** .5
)
order1 = [x[0] for x in order1]
order2 = sorted(prevalence.items(), 
    key= lambda t: -np.log2(t[1]) / len(t[0]),
)
order2 = [x[0] for x in order2]

In [21]:
order2

[((2, 'L'),),
 ((2, 'R'),),
 ((2, 'L'), (3, 'L'), (4, 'L')),
 ((0, 'L'), (1, 'L'), (2, 'L')),
 ((2, 'L'), (4, 'L')),
 ((2, 'L'), (4, 'R')),
 ((1, 'R'), (2, 'L'), (4, 'R')),
 ((1, 'L'), (2, 'L')),
 ((1, 'R'), (2, 'L')),
 ((1, 'R'), (2, 'L'), (4, 'L')),
 ((2, 'L'), (3, 'R'), (4, 'L')),
 ((1, 'L'), (2, 'L'), (4, 'R')),
 ((1, 'L'), (2, 'L'), (4, 'L')),
 ((0, 'L'), (2, 'L'), (4, 'L')),
 ((0, 'R'), (1, 'L'), (2, 'L')),
 ((1, 'L'), (2, 'L'), (3, 'L')),
 ((0, 'L'), (2, 'L')),
 ((0, 'R'), (2, 'L')),
 ((0, 'L'), (2, 'L'), (4, 'R')),
 ((2, 'L'), (3, 'L')),
 ((2, 'L'), (3, 'R')),
 ((0, 'R'), (2, 'L'), (3, 'R')),
 ((0, 'R'), (2, 'L'), (4, 'L')),
 ((1, 'L'), (2, 'L'), (3, 'R')),
 ((1, 'R'), (2, 'L'), (3, 'L')),
 ((0, 'R'), (2, 'L'), (4, 'R')),
 ((0, 'L'), (2, 'L'), (3, 'R')),
 ((0, 'R'), (2, 'L'), (3, 'L')),
 ((1, 'R'), (2, 'L'), (3, 'R')),
 ((0, 'L'), (2, 'L'), (3, 'L')),
 ((0, 'L'), (1, 'R'), (2, 'L')),
 ((0, 'L'), (1, 'L')),
 ((3, 'L'), (4, 'L')),
 ((1, 'R'), (4, 'R')),
 ((1, 'R'), (4, 'L')),
 ((

In [19]:
print([(i, x, y) for i, (x, y) in enumerate(zip(order1, order2)) if x != y])

[(4, ((1, 'R'), (2, 'L'), (4, 'R')), ((2, 'L'), (4, 'L'))), (5, ((2, 'L'), (4, 'L')), ((2, 'L'), (4, 'R'))), (6, ((2, 'L'), (4, 'R')), ((1, 'R'), (2, 'L'), (4, 'R'))), (7, ((1, 'R'), (2, 'L'), (4, 'L')), ((1, 'L'), (2, 'L'))), (8, ((1, 'L'), (2, 'L')), ((1, 'R'), (2, 'L'))), (9, ((1, 'R'), (2, 'L')), ((1, 'R'), (2, 'L'), (4, 'L'))), (16, ((0, 'L'), (2, 'L'), (4, 'R')), ((0, 'L'), (2, 'L'))), (17, ((0, 'R'), (2, 'L'), (3, 'R')), ((0, 'R'), (2, 'L'))), (18, ((0, 'L'), (2, 'L')), ((0, 'L'), (2, 'L'), (4, 'R'))), (19, ((0, 'R'), (2, 'L')), ((2, 'L'), (3, 'L'))), (20, ((0, 'R'), (2, 'L'), (4, 'L')), ((2, 'L'), (3, 'R'))), (21, ((1, 'L'), (2, 'L'), (3, 'R')), ((0, 'R'), (2, 'L'), (3, 'R'))), (22, ((1, 'R'), (2, 'L'), (3, 'L')), ((0, 'R'), (2, 'L'), (4, 'L'))), (23, ((2, 'L'), (3, 'L')), ((1, 'L'), (2, 'L'), (3, 'R'))), (24, ((2, 'L'), (3, 'R')), ((1, 'R'), (2, 'L'), (3, 'L'))), (49, ((1, 'R'), (3, 'R')), ((4, 'L'),)), (50, ((0, 'L'), (3, 'L')), ((4, 'R'),)), (51, ((0, 'L'), (1, 'R')), ((1, '

In [4]:
y_pred

[((2, 'L'),),
 ((2, 'R'),),
 ((2, 'L'), (3, 'L'), (4, 'L')),
 ((0, 'L'), (1, 'L'), (2, 'L')),
 ((2, 'L'), (4, 'L')),
 ((2, 'L'), (4, 'R')),
 ((1, 'R'), (2, 'L'), (4, 'R')),
 ((1, 'R'), (2, 'L'), (4, 'L')),
 ((1, 'L'), (2, 'L')),
 ((1, 'R'), (2, 'L')),
 ((0, 'L'), (2, 'L')),
 ((0, 'R'), (2, 'L')),
 ((0, 'L'), (2, 'L'), (4, 'L')),
 ((2, 'L'), (3, 'R'), (4, 'L')),
 ((1, 'L'), (2, 'L'), (4, 'R')),
 ((0, 'L'), (2, 'L'), (4, 'R')),
 ((0, 'R'), (2, 'L'), (3, 'R')),
 ((1, 'L'), (2, 'L'), (4, 'L')),
 ((1, 'L'), (2, 'L'), (3, 'L')),
 ((2, 'L'), (3, 'L')),
 ((2, 'L'), (3, 'R')),
 ((0, 'R'), (2, 'L'), (4, 'L')),
 ((0, 'L'), (2, 'L'), (3, 'R')),
 ((0, 'R'), (2, 'L'), (3, 'L')),
 ((0, 'R'), (2, 'L'), (4, 'R')),
 ((0, 'R'), (1, 'L'), (2, 'L')),
 ((0, 'L'), (1, 'R'), (2, 'L')),
 ((1, 'R'), (2, 'L'), (3, 'L')),
 ((1, 'L'), (2, 'L'), (3, 'R')),
 ((0, 'L'), (2, 'L'), (3, 'L')),
 ((1, 'R'), (2, 'L'), (3, 'R')),
 ((2, 'L'), (3, 'L'), (4, 'R')),
 ((0, 'L'), (1, 'L')),
 ((3, 'L'), (4, 'L')),
 ((1, 'R'), (4, 